## 1 Data loading Via Google Drive

In [0]:
##### Import data via Google Drive Mount

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


## 1.1 Basic Directory Loading and Setting Seeds

In [0]:
from numpy.random import seed
seed(137)
from tensorflow import set_random_seed
set_random_seed(191)

# Dir
train_dir = "/gdrive/My Drive/newCircleData/Train"
val_dir = "/gdrive/My Drive/newCircleData/Val"
test_dir = "/gdrive/My Drive/newCircleData/Test"


img_width = 600
img_height = 450
batch_size = 5
channels = 3

## 1.2 Data loading via Keras DataImageGenerator

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)             
val_datagen = ImageDataGenerator(rescale=1./255)              

train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True,
        class_mode='binary')   

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    class_mode='binary') #weight toward one class or another

#Keras takes care of generating labels if the directory structure matches above!
label_mapT = train_generator.class_indices
print(label_mapT)

label_mapV = validation_generator.class_indices
print(label_mapV)

for data_batch, labels_batch in train_generator:
    print ('data batch shape:', data_batch.shape)
    #print(data_batch)
    print('labels batch shape:', labels_batch.shape)
    #print(labels_batch)
    break
    
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)

Using TensorFlow backend.


Found 395 images belonging to 2 classes.
Found 145 images belonging to 2 classes.
{'G': 0, 'S': 1}
{'G': 0, 'S': 1}
data batch shape: (5, 450, 600, 3)
labels batch shape: (5,)


## 2 Model training

In [0]:
import numpy as np 
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Lambda
from keras.metrics import categorical_accuracy
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential

from keras.callbacks import TensorBoard

#from keras.layers import Input, Dense
from keras.models import Model

input_shape = (img_height, img_width, channels)

model = Sequential()

model.add(Conv2D(32,(11, 11), input_shape=input_shape))#, kernel_regularizer=regularizers.l1(0.01))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32,(3, 3), input_shape=input_shape))#, kernel_regularizer=regularizers.l1(0.01))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32,(3, 3), input_shape=input_shape))#, kernel_regularizer=regularizers.l1(0.01))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5)) 
model.add(Dense(1))
model.add(Activation('sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 440, 590, 32)      11648     
_________________________________________________________________
activation_1 (Activation)    (None, 440, 590, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 220, 295, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 220, 295, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 218, 293, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 218, 293, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 109, 146, 32)      0         
__________

In [0]:
# For early stopping
import keras
from keras.callbacks import TensorBoard, Callback, EarlyStopping

class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)
        
callbacks_list = [EarlyStopping(monitor='val_acc', patience=10, verbose=1)]

In [0]:
from keras import optimizers

epochs = 50

opt = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.1)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    shuffle=True,
    callbacks=callbacks_list+[MetricsCheckpoint('logs')])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
79/79 [==============================] - 195s 2s/step - loss: 3.2466 - acc: 0.7823 - val_loss: 0.8522 - val_acc: 0.9448
Epoch 2/50
79/79 [==============================] - 15s 192ms/step - loss: 1.3744 - acc: 0.9063 - val_loss: 0.5497 - val_acc: 0.9655
Epoch 3/50
79/79 [==============================] - 16s 200ms/step - loss: 1.0325 - acc: 0.9316 - val_loss: 0.5497 - val_acc: 0.9655
Epoch 4/50
79/79 [==============================] - 16s 200ms/step - loss: 1.3943 - acc: 0.9089 - val_loss: 0.5497 - val_acc: 0.9655
Epoch 5/50
79/79 [==============================] - 16s 199ms/step - loss: 1.1514 - acc: 0.9215 - val_loss: 0.4659 - val_acc: 0.9655
Epoch 6/50
79/79 [==============================] - 16s 199ms/step - loss: 0.9719 - acc: 0.9392 - val_loss: 0.4495 - val_acc: 0.9655
Epoch 7/50
79/79 [==============================] - 16s 200ms/step - loss: 1.1277 - acc: 0.9266 - val_loss: 0.5497 - val_acc: 0.9655
Epoch 8/50
79/79 [=====

## 3 Prediction

In [0]:
#Predicting labels of test set using OCT-trained CNN + Random Forest Classifier
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        color_mode="rgb",
        class_mode="binary",
        shuffle=False,
        batch_size=batch_size)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator, steps = np.ceil(nb_samples / batch_size))
    
#model.predict_proba()  
# Getting binary prediction
#predict = (predict)
print(predict)

#model.evaluate_generator(test_generator, steps = np.ceil(nb_samples / batch_size))

#model.predict(test_generator)

Found 197 images belonging to 2 classes.
[[1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [2.3752516e-25]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [1.000

In [0]:
#Getting false positives and false negatives
labels = test_generator.classes
predictions = predict
print(labels. reshape(197, 1))
print(labels.shape)
print(predictions.shape)

predictions = predictions.reshape(len(labels))

print(predictions.transpose())

FP_list = []
FN_list = []

#FP
for i in range(len(labels)):
  if labels[i] == 0 and predictions[i] == 1:
    FP_list.append(filenames[i])

#FN
for i in range(len(labels)):
  if labels[i] == 1 and predictions[i] == 0:
    FN_list.append(filenames[i])

print(len(FP_list))
print(len(FN_list))
print(FP_list)
print(FN_list)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
(197,)
(197, 1